In [3]:
import PIL
import numpy as np
import torch
import matplotlib.pyplot as plt

import os
import json

import cv2

In [4]:
KPS_MAPPING = [49, 8, 57, 0, 52, 5, 11, 7, 20, 23, 24, 33, 25, 32, 28,
               29, 46, 34, 37, 50, 65, 64, 9, 48]

with open("../mesh/baoshijie-kayan_keypts.txt","r") as file:
    kpts_66 = []
    
    for line in file.readlines():
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        x,y,score = line.split()
        kpts_66.append(line.split())

with open("../mesh/baoshijie-kayan_24keypts.txt","w") as file:
    kpts_66_ndarray = np.array(kpts_66)
    kpts_24_ndarray = kpts_66_ndarray[KPS_MAPPING]
    kpts_24 = kpts_24_ndarray.tolist()

    for kpt in kpts_24:
        file.write(f"{kpt[0]} {kpt[1]} {kpt[2]}\n")

In [33]:
image_folder="./images/"
image_name = "sample.jpg"

kp_threshold = 0.1

pil_im = PIL.Image.open(image_folder+image_name).convert('RGB')


fx = 2304.5479
fy = 2305.8757
cx = 1686.2379
cy = 1354.9849
camera_matrix = np.array([[fx,0,  cx],
                          [0, fy, cy],
                          [0, 0,  1]], dtype=np.float32)

dist_coeffs = np.zeros((4, 1)) 

In [3]:

os.system("python -m openpifpaf.predict " + image_folder + image_name + \
  " --checkpoint=shufflenetv2k16-apollo-24 --json-output ./json \
  --instance-threshold 0.05 --seed-threshold 0.05 \
  --line-width 4 --font-size 0")

with open("./json/"+image_name+".predictions.json") as file:
    vehicles = json.load(file)

In [ ]:
keypoint_list

In [46]:
first_car = vehicles[2]

keypoint_list = first_car['keypoints']

visable_index = []
visable_keypoints_2D = []

for i in range(0,24):
    x = keypoint_list[i*3]
    y = keypoint_list[i*3+1]
    score = keypoint_list[i*3+2]

    if(score > kp_threshold):
        visable_index.append(i)
        visable_keypoints_2D.append([x,y])

visable_keypoints_2D

[[1710.09, 780.16],
 [1597.91, 918.48],
 [1712.12, 965.97],
 [1755.12, 912.15],
 [1758.08, 786.36],
 [1901.88, 785.46],
 [1806.89, 882.64],
 [1629.49, 840.18]]

In [47]:
with open("./mesh/dazhong_24keypts.txt") as file:
    keypoints_3D_24 = []
    
    for line in file.readlines():
        line = line.strip('\n')  #去掉列表中每一个元素的换行符
        x,y,score = line.split()
        keypoints_3D_24.append(line.split())

keypoints_3D_24_ndarray = np.array(keypoints_3D_24)
visable_keypoints_3D = keypoints_3D_24_ndarray[visable_index].tolist()

visable_keypoints_3D

[['-0.64906948', '-0.61638962', '-0.16350945'],
 ['-0.78735706', '0.34957592', '1.43293312'],
 ['-0.78735706', '0.34957607', '-1.30198021'],
 ['-0.80440891', '-0.15282092', '-1.92037194'],
 ['-0.60184902', '-0.53747243', '-1.28940911'],
 ['0.60395638', '-0.53739655', '-1.28956475'],
 ['-0.58979885', '-0.24539752', '-2.23300560'],
 ['-0.80719463', '-0.21388827', '0.93815147']]

In [48]:


objPoints = np.array(visable_keypoints_3D,dtype='float64')
imgPoints = np.array(visable_keypoints_2D,dtype='float64')

(_, rotation_vector, translation_vector) = cv2.solvePnP(objPoints,
                                                        imgPoints,
                                                        camera_matrix,
                                                        dist_coeffs,
                                                        useExtrinsicGuess=False,flags=cv2.SOLVEPNP_UPNP
                                                        )

In [49]:
rotation_vector

array([[ 0.30603494],
       [-0.2410071 ],
       [-0.09093339]])

In [50]:
translation_vector

array([[ 0.59826905],
       [-2.8863501 ],
       [13.8660819 ]])

In [51]:
with open("./mesh/dazhong.json") as json_file:
        car_model = json.load(json_file)

vertices = np.array(car_model['vertices'])  # x: w y: h z: l
triangles = np.array(car_model['faces']) - 1


(projected_point2D_model, jacobian_model) = cv2.projectPoints(vertices, rotation_vector, translation_vector,camera_matrix, dist_coeffs)

In [52]:
def draw_obj(image, vertices, triangles, color):
    for t in triangles:
        coord = np.array(
            [vertices[t[0]][:2], vertices[t[1]][:2],
                vertices[t[2]][:2]], dtype=np.int32
        )
        cv2.polylines(image, np.int32([coord]), 1, color)
        
def draw_obj_mask(image, vertices, triangles, color):
    for t in triangles:
        coord = np.array(
            [vertices[t[0]][:2], vertices[t[1]][:2],
                vertices[t[2]][:2]], dtype=np.int32
        )
        cv2.fillPoly(image, np.int32([coord]),  color)

im = np.asarray(pil_im)
im = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
draw_obj(im,projected_point2D_model,triangles,(0, 125, 255))

cv2.imwrite("draw_obj.jpg",im)

True